In [3]:
import pandas as pd
pd.options.display.max_rows=1000
import numpy as np

# Load and format data

In [4]:
df = pd.read_excel('/Users/nathansuberi/Downloads/retailreport2.xls')
df = df.reset_index()
df = df.dropna(axis=0, how='all')
df = df.drop([1,6,8,10])
df.columns = df.loc[12]
df = df.drop([12,15])
df = df.dropna(axis=1, how='all')

In [5]:
df.columns = ['DCS Code',
 'Vendor\nCode',
 'Description1',
 None,
 'Attribute',
 None,
 'Size',
 'UPC',
 'QTY',
 None,
 'Ext Price',
 'Ext Cost',
 'Ext Margin Amount',
 'Margin %']

df.drop(None, axis=1)

,DCS Code,Vendor Code,Description1,Attribute,Size,UPC,QTY,Ext Price,Ext Cost,Ext Margin Amount,Margin %
17,W C,BOR,JULIANA BORN,BLK,8.5,NaN,1.0,90.00,39.50,50.50,56.11
19,W S,KEE,ROSE SANDAL,GARG/RAV,9.5,NaN,1.0,90.00,45.00,45.00,50.00
21,W C,DAN,PRO PATENT,SILBLUTO,37,NaN,1.0,135.00,65.00,70.00,51.85
23,W C,DAN,PRO PATENT,SILBLUTO,40,NaN,1.0,125.00,65.00,60.00,48.00
25,W B,PIK,PIK 838-8527C1,NAVY,41,NaN,1.0,140.00,95.00,45.00,32.14
27,W C,DAN,MAUREEN DAN,BLK,39,NaN,1.0,130.00,59.80,70.20,54.00
29,W C,DAN,MAUREEN DAN,BLK,40,NaN,1.0,130.00,59.80,70.20,54.00
31,W C,DAN,MAUREEN DAN,BLK,38,NaN,1.0,130.00,59.80,70.20,54.00
33,W C,DAN,MAUREEN DAN,BLK,37,NaN,1.0,130.00,59.80,70.20,54.00
34,W C,DAN,MAUREEN DAN,BLK,42,NaN,1.0,130.00,59.80,70.20,54.00


In [7]:
trimmed_df = df.drop([12192, 12194, 12197])

In [8]:
sandal_flag = [a or b for a,b in zip(trimmed_df['DCS Code'].str.contains('W  S'), trimmed_df['DCS Code'].str.contains('M  S'))]
sandals = df.loc[sandal_flag]

In [9]:
sandals

,DCS Code,Vendor Code,Description1,None,Attribute,None,Size,UPC,QTY,None,Ext Price,Ext Cost,Ext Margin Amount,Margin %
19,W S,KEE,ROSE SANDAL,NaN,GARG/RAV,NaN,9.5,NaN,1.0,NaN,90.00,45.00,45.00,50.00
36,W S,DAN,SEASON DANSKO,NaN,WHITE,NaN,42,NaN,1.0,NaN,120.00,57.50,62.50,52.08
40,M S,BIR,ARIZ EVA BLK 12942,NaN,1,NaN,43,NaN,1.0,NaN,39.95,20.00,19.95,49.94
60,W S,FIN,CATALINA SF FINN,NaN,BLK P,NaN,35,NaN,1.0,NaN,225.00,105.00,120.00,53.33
62,W S,ORT,PACIFIC HIGHTIDE,NaN,PEWT,NaN,10.0,NaN,1.0,NaN,69.95,34.00,35.95,51.39
110,W S,NAO,ODELIA NAOT,NaN,BLUE/EME,NaN,38,NaN,0.0,NaN,0.00,0.00,0.00,0.00
112,W S,MEP,HELEN PLUS WIDE,NaN,7103,NaN,39,NaN,1.0,NaN,149.00,68.50,80.50,54.03
114,W S,OLU,OHANA OLUKAI WOMENS,NaN,DAHLIA,NaN,6.0,NaN,1.0,NaN,65.00,31.00,34.00,52.31
146,W S,NAO,DORITH NAOT,NaN,GRECIANG,NaN,38,NaN,1.0,NaN,130.00,65.00,65.00,50.00
214,W S,NAO,KAYLA NAOT,NaN,WHITE,NaN,38,NaN,1.0,NaN,135.00,62.50,72.50,53.70


In [10]:
grouped = df.groupby(['Vendor\nCode','Description1', 'Attribute'])
sums = grouped.sum()
vendors = list(sums.index.levels[0])

# For output to HTML

In [ ]:
with open('/Users/nathansuberi/Desktop/enchanted_reports/summer_sale_2018/summer_sale_reference.html', 'w') as f:
    for vendor in vendors:
        vendor_data = sums.loc[vendor, :, :].reset_index()
        sale_flag = vendor_data['QTY'] < 5
        on_sale = vendor_data.loc[sale_flag, ['Description1', 'Attribute', 'QTY']].reset_index().drop('index', axis=1)
        not_on_sale = vendor_data.loc[~sale_flag, ['Description1', 'Attribute', 'QTY']].reset_index().drop('index', axis=1)
        
        if not_on_sale.shape[0]:
            f.write('<h1>Vendor: ' + vendor + '</h1>\n')
            last_style = ''
            for line in not_on_sale.values:
                style = line[0]
                color = line[1]
                qty = int(line[2])
                if last_style == style:
                    f.write('<i>{}, {}, sold: <font size=4>{}</font>'.format(style, color, qty) + '</i> <br/>')
                else:
                    f.write('<br/><b>{}, {}, sold: <font size=4>{}</font>'.format(style, color, qty) + '</b> <br/>')
                last_style = style

# For output to csvs

In [12]:
for vendor in vendors:
    vendor_data = sums.loc[vendor, :, :].reset_index()
    sale_flag = vendor_data['QTY'] < 4
    on_sale = vendor_data.loc[sale_flag, ['Description1', 'Attribute', 'QTY']].reset_index().drop('index', axis=1)
    not_on_sale = vendor_data.loc[~sale_flag, ['Description1', 'Attribute', 'QTY']].reset_index().drop('index', axis=1)

    if on_sale.shape[0]:
        on_sale.to_csv('/Users/nathansuberi/Desktop/enchanted_reports/summer_sale_2018/onsale_{}.csv'.format(vendor), header=None)
    if not_on_sale.shape[0]:
        not_on_sale.to_csv('/Users/nathansuberi/Desktop/enchanted_reports/summer_sale_2018/notonsale_{}.csv'.format(vendor), header=None)